In [1]:
## Installing Required Python Libraries

!pip install markdown
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install pypdf
!pip install pandas
!pip install sentence-transformers
!pip install chromadb
!pip install text_generation

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not write

In [2]:
import warnings

# Ignore specific UserWarning
warnings.filterwarnings("ignore", message="Field \"model_id\" has conflict with protected namespace \"model_\".")

In [3]:
## Import Required Python Libraries

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
#from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
#from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken
import markdown
import pandas as pd
import sentence_transformers

import os

In [5]:
## Loading Sec Specification PDFs

# from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders.pdf import PyPDFLoader, PyPDFDirectoryLoader
pdf_folder_path = 'ragsecops/data/specifications'

loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()

In [6]:
## Check the first lines of the raw content document

import pandas as pd

df = pd.DataFrame(docs)
print(df.head())

                                                   0  \
0  (page_content, Common\nVulnerability\nScoring\...   
1  (page_content, Contents\n1.\nIntroduction\n3\n...   
2  (page_content, 8.\nCVSS\nv4.0\nEquations\n33\n...   
3  (page_content, 1.\nIntroduction\nThe\nCommon\n...   
4  (page_content, Figure\n1:\nCVSS\nMetric\nGroup...   

                                                   1                 2  
0  (metadata, {'source': 'ragsecops/data/specific...  (type, Document)  
1  (metadata, {'source': 'ragsecops/data/specific...  (type, Document)  
2  (metadata, {'source': 'ragsecops/data/specific...  (type, Document)  
3  (metadata, {'source': 'ragsecops/data/specific...  (type, Document)  
4  (metadata, {'source': 'ragsecops/data/specific...  (type, Document)  


In [81]:
## Split the data in chunks large enough to have meaningful answers, and some overlap not to miss anything

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 40)
all_splits = text_splitter.split_documents(docs)

In [82]:
## Store the data as embeddings in a vector database (Chroma or FAISS)
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=all_splits, embedding=HuggingFaceEmbeddings())

#vectorstore = Chroma.from_documents(documents=all_splits, embedding=HuggingFaceEmbeddings())


In [65]:
question = "Impact Metrics in CVSS"
docs = vectorstore.similarity_search(question)
docs

[Document(page_content='metric\nvalue\n(Conﬁdentiality\nHigh).\nWhen\nidentifying\nvalues\nfor\nthe\nimpact\nmetrics,\nassessment\nproviders\nneed\nto\naccount\nfor \nimpacts\nboth\nto\nthe\nVulnerable\nSystem\nand\nimpacts\noutside\nof\nthe\nVulnerable\nSystem.\nThese \nimpacts\nare\nestablished\nby\ntwo\nsets\nof', metadata={'source': 'ragsecops/data/specifications/cvss-v40-specification.pdf', 'page': 12}),
 Document(page_content='but\nalso\nincludes\nhuman\nsafety.\nThis\npotential\nfor\nmeasuring\nthe\nimpact\nof\na \nvulnerability\nother\nthan\nthe\nvulnerable\nsystem,\nwas\na\nkey\nfeature\nintroduced\nwith\nCVSS\nv3.0. \nThis\nproperty\n(formerly\nknown\nas\n“Scope”),\nis\ncaptured\nby\nthe\nseparation\nof\nimpacts', metadata={'source': 'ragsecops/data/specifications/cvss-v40-specification.pdf', 'page': 4}),
 Document(page_content='should\nreference\nthe\n“end \ngame”\nImpact\nmetric\nvalue', metadata={'source': 'ragsecops/data/specifications/cvss-v40-specification.pdf', 'page':

In [83]:
## Create the Chain

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

## Define the Prompt
## https://www.promptingguide.ai/models/mistral-7b#chat-template-for-mistral-7b-instruct
template="""<s>[INST] Instruction [/INST]
You are a helpful, respectful, and honest assistant designed to provide answers to questions based on the provided context. Your responses should be informative, safe, and based on the information given in the context. Avoid sharing any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Ensure your responses are socially unbiased and positive.

If the question does not make sense, is not factually coherent, or if you do not have enough information to provide a correct answer, explain the situation instead of providing incorrect or misleading information.

Question: {question}
Context: {context} [/INST]

Model answer

<s>[INST] Follow-up instruction [/INST]
When providing follow-up information or answering additional questions, continue to adhere to the guidelines mentioned above. Ensure your responses remain helpful, accurate, and based on the context provided.
[/INST]
"""


In [84]:
## Using Mistral instead of Llama2
inference_server_url="http://localhost:8080"

In [85]:
## Define the QA Chain Prompt and the HFTGI llm

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.1,
    repetition_penalty=1.175,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [86]:
## Initialize a question-answering chain with LLM and vector-based document retrieval, 
## including source documents in the response.

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectorstore.as_retriever(),
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True)

In [88]:
question = "What is CVSS"

result = qa_chain({"query": question})


Follow-up model answer:

To further elaborate, CVSS (Common Vulnerabilities Scoring System) is an industry-standard tool used for measuring the potential impact of vulnerabilities on a system. It was introduced with CVSS v3.0 as a key feature that captures the separation of impacts into metrics. This property allows assessment providers to record or transfer CVSS metric information using a metric value (Confidentiality High). When identifying values for impact metrics, it's crucial to consider both the effects on the vulnerable system and external impacts. These impacts are established through two sets of end game Impact metric value references.

In [90]:
question = "What are the Attack Requirements (AT)"

result = qa_chain({"query": question})


Follow-up model answer:

The Attack Requirements (AT) refer to the minimum level of resources needed to exploit a vulnerability. This includes both technical requirements and human factors such as skill level and motivation. The CVSS v3.0 introduced this property, known as "Attacker Complexity," which captures the complexity of an attacker's ability to execute an attack. It ranges from Low to High, with High indicating that the attack requires significant expertise and resources. When assessing impact metrics, it is crucial to consider both the effects on the vulnerable system and external impacts. These are measured using two sets of metrics, typically Confidentiality, Integrity, Availability, and User Privileges.

In [102]:
question = "What is Exploit Maturity"

result = qa_chain({"query": question})


Follow-up model answer:

To further elaborate, exploit maturity refers to the potential for measuring the impact of a vulnerability beyond just the vulnerable system itself. This property was introduced with CVSS v3.0 as a key feature, allowing assessment providers to account for impacts both within and outside the Vulnerable System. The two sets of end game Impact metric values establish these external impacts, which can then be recorded or transferred along with CVSS metric information.